## Program to anonymize A and B numbers prior to analysis
##### 1) Input file created with required fields
##### 2) Use Faker Factory & Default Dict to generate fake numbers for A & B
##### 3) Reset indexes
##### 4) Write to file
 

### Import required libraries

In [10]:
import pandas as pd
from faker import Factory
from collections import defaultdict

### Import file data into pandas dataframe

In [43]:
sampledata=pd.read_csv("C:\\Users\\181916\\SNA\\ShortFile",sep='|')
sampledata.head()

,Unnamed: 0,AP_IMS,AP_NUM,AP_IGN,BP_IMS,BP_NUM,BP_IGN,CALL_DT,CALL_TM,CALL_SEC,CALL_MSEC
0,0,NaN,18711236764,NaN,2.400706e+14,561739037,3.567891e+14,20191130,175035,6.0,0
1,1,NaN,18711236764,NaN,2.400706e+14,561739037,3.567891e+14,20191130,174954,43.0,0
2,2,2.400786e+14,49139046764,3.553451e+14,NaN,1615437370,NaN,20191130,234357,40.0,0
3,3,2.400785e+14,91699280764,3.528171e+14,2.400782e+14,0583510270,NaN,20191130,224956,3677.0,0
4,4,2.400782e+14,91883700764,3.567651e+14,2.400782e+14,2007459370,NaN,20191130,234254,275.0,0


### Create Faker Factory

In [12]:

faker=Factory.create()

### Generate masked numbers
#### 1) Combined list of A_NUM and B_NUM.
#### 2) Dedup
#### 3) Rename col name to Number
#### 4) Use default dict to generate masked number for each number

In [38]:
numlist=[]
numset=set(list(sampledata['AP_NUM'])+list(sampledata['BP_NUM']))
numlist=list(numset)
allnumlist=pd.DataFrame(numlist)
allnumlist.columns=['NUMBER']
list(allnumlist.columns)

['NUMBER']

In [39]:
allnumlist['MASKNUM']=allnumlist['NUMBER'].map(defaultdict(faker.phone_number))

#### Merge the maske dnumber with main data frame - first for A Num then B Num
#### Drop extra columns 
#### copy to new DF

In [50]:
merged_dataframe = pd.merge(sampledata, allnumlist, left_on= "AP_NUM", right_on= "NUMBER", how= "inner")
merge1df=merged_dataframe[['BP_NUM','CALL_DT','CALL_TM','CALL_SEC','CALL_MSEC','MASKNUM']]
#merged_dataframe = pd.merge(sampledata, allnumlist, left_on= "BP_NUM", right_on= "NUMBER", how= "inner")
merge1df.columns=['BP_NUM','CALL_DT','CALL_TM','CALL_SEC','CALL_MSEC','APMASKNUM']
merged_dataframe=pd.merge(merge1df,allnumlist,left_on="BP_NUM",right_on="NUMBER",how="inner")
merge2df=merged_dataframe[['APMASKNUM','MASKNUM','CALL_DT','CALL_TM','CALL_SEC','CALL_MSEC']]
#merged_dataframe = pd.merge(sampledata, allnumlist, left_on= "BP_NUM", right_on= "NUMBER", how= "inner")
merge2df.columns=['APMASKNUM','BPMASKNUM','CALL_DT','CALL_TM','CALL_SEC','CALL_MSEC']
merge2df.head()


#sampledata['AP_NUM_MASK']=sampledata['AP_NUM'].map(allnumlist)

,APMASKNUM,BPMASKNUM,CALL_DT,CALL_TM,CALL_SEC,CALL_MSEC
0,5552672684,(639)663-8007x6286,20191130,175035,6.0,0
1,5552672684,(639)663-8007x6286,20191130,174954,43.0,0
2,518-294-4233x93153,(416)606-6912x60695,20191130,234357,40.0,0
3,(773)933-2121,(629)392-9569x1627,20191130,224956,3677.0,0
4,753-664-9496x027,(656)700-3467x92491,20191130,234254,275.0,0


In [51]:
anondata=merge2df.copy()

### Reset index

In [16]:
anondata=sampledata[['AP_NUM_MASK','BP_NUM_MASK','CALL_DT','CALL_TM','CALL_SEC','CALL_MSEC']]
anondata.reset_index(drop=True,inplace=True)

### Verify Data

In [17]:
anondata.head()


,AP_NUM_MASK,BP_NUM_MASK,CALL_DT,CALL_TM,CALL_SEC,CALL_MSEC
0,+1-584-486-5733,(311)744-4072,20191130,175035,6.0,0
1,+1-584-486-5733,(311)744-4072,20191130,174954,43.0,0
2,385-896-5513,+1-174-987-4923x6930,20191130,234357,40.0,0
3,142.054.4797,519-666-5196x8090,20191130,224956,3677.0,0
4,831.135.5363,342.624.9883,20191130,234254,275.0,0


### Write data to file and close

In [52]:
anondata.to_csv("C:\\Users\\181916\\SNA\ShortFileAnon",sep='|')

### Anonymized data ready to use